# Course: **COMP 333 — Final Project Phase 1**
**Group:** O <br>
**Names:**
- Carson Johnston **Student ID:** 40312846
- Charlotte Lauzon **Student ID:** 40285642
- Ava Samimi **Student ID:** 40048117

## Description of the task:


## Source of the datasets:
The **US Used Cars** dataset is a public dataset available on Kaggle: <br> https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset

## Description of the datasets:
**The *US Used Cars* dataset contains:**

**The notebook will include: **


In [3]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.gridspec as gridspec
import warnings

# Avoid unnecessary warnings in output
warnings.filterwarnings("ignore")

# Doesn't cut tables
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

# Apply predefined visual theme for consistence in the export
plt.style.use('seaborn-v0_8')
sns.set_theme()

# Load the datasets
cars = pd.read_csv('used_cars_data.csv')
